In [ ]:
# Install necessary libraries
#!pip install --upgrade --force-reinstall "numpy<2.0.0" vllm numba tensorflow torch unsloth huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login

# Replace 'your_hf_token' with your actual Hugging Face access token
login(token="your_hf_token")


In [ ]:

from unsloth import FastLanguageModel, is_bfloat16_supported
from filelock import FileLock
from transformers import AutoTokenizer
import torch
import json
import os
from vllm import SamplingParams
from huggingface_hub import hf_hub_download

# Model configuration
max_seq_length = 2048
lora_rank = 64

# Download the Qwen-3B model directly from Hugging Face
model_name = "Qwen/Qwen2.5-3B-Instruct"  # or another model name
lora_adapter = "AMLAN69/LoRA"  # LoRA adapter for Qwen-3B Llama-1B-LoRA/blob/main/adapter_model%20(2).bin

# Load base model and LoRA adapter
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    fast_inference=True,
    max_lora_rank=lora_rank,
    lora_weight_path=hf_hub_download(lora_adapter, filename="adapter_model.bin"),
    gpu_memory_utilization=0.5,
)

def load_conversations(user_id, history_file="conversations.jsonl", max_history=10):
    """Load conversation history with file locking"""
    conversations = []
    if not isinstance(user_id, (str, int)):
        raise ValueError("Invalid user ID format")

    if os.path.exists(history_file):
        with FileLock(history_file + ".lock"):
            try:
                with open(history_file, "r", encoding="utf-8") as f:
                    for line in f:
                        data = json.loads(line)
                        if str(data["user_id"]) == str(user_id):
                            conversations.append(data)
            except Exception as e:
                print(f"Error loading history: {e}")

    return conversations[-max_history:]

def save_conversation(user_id, user_message, bot_response,
                     history_file="conversations.jsonl", max_history=5):
    """Save conversation with validation and locking"""
    if not isinstance(user_id, (str, int)):
        raise ValueError("Invalid user ID format")

    user_message = user_message.strip()[:2000]  # Limit message length
    bot_response = bot_response.strip()[:2000]

    new_entry = {
        "user_id": str(user_id),
        "user_message": user_message,
        "bot_response": bot_response
    }

    with FileLock(history_file + ".lock"):
        try:
            with open(history_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(new_entry) + "\n")
        except Exception as e:
            print(f"Error saving conversation: {e}")

def generate_response(user_id, user_message, model):
    """Generate response with safety checks"""
    # Input validation
    user_message = user_message.strip()[:2000]
    if not user_message:
        return "Please provide a valid message."

    SYSTEM_PROMPT = """You are a CBT-based mental health chatbot. Follow these rules:
1. Empathize first, then help identify negative thoughts
2. Use Socratic questioning to challenge cognitive distortions
3. Suggest practical behavioral activation strategies
4. Never provide medical advice
5. Maintain natural conversation flow"""

    try:
        past_conversations = load_conversations(user_id)
        chat_history = [{"role": "system", "content": SYSTEM_PROMPT}]

        # Build conversation history
        for conv in past_conversations:
            chat_history.append({"role": "user", "content": conv["user_message"]})
            chat_history.append({"role": "assistant", "content": conv["bot_response"]})

        chat_history.append({"role": "user", "content": user_message})

        # Format input for model
        text = tokenizer.apply_chat_template(
            chat_history,
            tokenize=False,
            add_generation_prompt=True
        )

        # Generation parameters
        sampling_params = SamplingParams(
            temperature=0.85,
            top_p=0.9,
            max_tokens=2048,
            stop=["<|endoftext|>", "USER:", "ASSISTANT:"]
        )

        # Generate response
        outputs = model.fast_generate(
            text,
            sampling_params=sampling_params,
        )
        response = outputs[0].outputs[0].text.strip()

        # Basic safety filter
        response = response.split("ASSISTANT:")[-1].split("USER:")[0].strip()

        # Save conversation
        save_conversation(user_id, user_message, response)
        return response

    except Exception as e:
        print(f"Error generating response: {e}")
        return "I'm having trouble responding right now. Please try again later."

"""CHATBOT INFERENCE"""

def chat_loop():
    user_id = "temp_user"  # You can make this dynamic if needed
    print("\nWelcome to the CBT Chatbot. Type 'exit' to end the conversation.\n")

    while True:
        try:
            # Get user input
            user_input = input("You: ")

            if user_input.lower() in ["exit", "quit"]:
                print("Ending conversation. Take care!")
                break

            # Generate and display response
            response = generate_response(user_id, user_input, model)
            print("\nBot:", response)
            print("---\n")

        except KeyboardInterrupt:
            print("\nConversation ended by user.")
            break
        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Please try rephrasing your message.")

# Start the chat
if __name__ == "__main__":
    chat_loop()


adapter_model.bin:   0%|          | 0.00/180M [00:00<?, ?B/s]

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 6.16 GB. Also swap space = 2 GB.
WARNING 04-05 05:42:35 [config.py:2614] Casting torch.bfloat16 to torch.float16.
INFO 04-05 05:42:51 [config.py:585] This model supports multiple tasks: {'reward', 'embed', 'generate', 'sco

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...


Bot: I'm not capable of executing code directly. I'm a text-based conversational AI, and my capabilities are focused on text-based input and output.

However, I can provide you with examples of how to write simple Python code if you're interested in doing so. If you're feeling stuck or wanting to explore coding concepts, I can offer guidance and resources to get you started.

What would you like to do? Do you want to learn about a specific topic, such as programming fundamentals, data structures, or something else?
---

You: tell me about python


Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.42s/it, est. speed input: 128.62 toks/s, output: 79.51 toks/s]



Bot: Python is a high-level, interpreted programming language known for its simplicity, readability, and ease of use. Here are some key features and concepts that make Python a popular choice for beginners and experts alike:

**Key Features:**

1. **Indentation-based syntax**: Python uses indentation to define code blocks, which makes it easier to read and understand.
2. **Object-Oriented Programming (OOP)**: Python supports OOP concepts like classes, objects, and inheritance, which make it easy to build complex programs.
3. **Dynamic typing**: Python is dynamically typed, which means you don't need to declare variable types before using them.
4. **Extensive libraries and frameworks**: Python has a vast collection of libraries and frameworks that make it easy to work with various tasks, such as web development, data analysis, and more.

**Programming Concepts:**

1. **Variables**: Variables are used to store and manipulate data. In Python, you can declare variables using the `var` key

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.48it/s, est. speed input: 4432.78 toks/s, output: 41.95 toks/s]



Bot: I can't provide a code that adds two numbers.
---


Conversation ended by user.
